# Machine Learning 과정
- 문제정의
- 데이터 수집
- 데이터 전처리 (인코딩,특성공학)
- 탐색적 데이터 분석 (시각화, 특성선택)
- 모델 선택 및 학습
- 하이퍼파라미터 튜닝 (교차검증,그리드서치)
- 모델 평가

In [ ]:
%matplotlib inline

import pandas as pd
pd.set_option('display.max_rows',None) # 데이터프레임의 최대 행 개수 해제
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv('./data/train.csv')

In [ ]:
data.shape

In [ ]:
data.head()

### 데이터 사전
- PassengerId : 탑승객 id
- Survived : 생존여부 (0 = 사망, 1 = 생존)
- Pclass : 티켓 클래스 (1등급, 2등급, 3등급)
- Name : 이름
- Sex : 성별 (male = 남성, female = 여성)
- Age : 나이
- SibSp : 형제/배우자의 수
- Parch : 부모/자녀 수
- Ticket : 티켓 번호
- Fare : 요금
- Cabin : 객실번호
- Embarked : 승선항 C = 쉘 부르그, Q = 퀸즈타운, S = 사우스 햄튼

In [ ]:
data.info()

비어있는 나이는 중간 값으로 채우자 (중간 값으로 하면 이상치에 대해 보완가능)

In [ ]:
data['Age'] = data['Age'].fillna(data['Age'].median())

In [ ]:
data.info()

시각화 편의를 위해 Died 컬럼 추가

In [ ]:
data['Died'] = 1 - data['Survived']

### 성별과의 관계

In [ ]:
data.groupby('Sex').sum()[['Survived', 'Died']]

In [ ]:
data.groupby('Sex').mean()[['Survived', 'Died']].plot(kind='bar', figsize=(25, 7), 
                                                           stacked=True, color=['g', 'r']);

남성이 많이 죽고 여성이 많이 살았다. (성별 컬럼은 생존과 관계가 많다)

### 나이와의 관계

In [ ]:
fig = plt.figure(figsize=(25, 7)) #그림 크기 지정
sns.violinplot(x='Sex', y='Age', #바이올린 그래프
               hue='Survived', data=data, 
               split=True,
               palette={0: "r", 1: "g"}
              );

- 남성 탑승객중 어린아이들이 조금 더 많이 살았다.
- 20~40 탑승객(남,여 모두)이 많이 사망했다.

### 요금과의 관계

In [ ]:
figure = plt.figure(figsize=(25, 7))
plt.hist([data[data['Survived'] == 1]['Fare'], data[data['Survived'] == 0]['Fare']], 
         stacked=True, color = ['g','r'],
         bins = 50, label = ['Survived','Dead'])
plt.xlabel('Fare')
plt.ylabel('Number of passengers')
plt.legend();

티켓 요금이 낮으면 많이 죽었다.

### 티켓요금과 선실등급과의 관계

In [ ]:
data.groupby('Pclass').mean()['Fare'].plot(kind='bar', figsize=(25, 7))

선실등급이 낮을 수록 티켓 가격이 싸다

### 승선항과의 관계

In [ ]:
fig = plt.figure(figsize=(25, 7))
sns.violinplot(x='Embarked', y='Fare', hue='Survived', data=data, split=True, palette={0: "r", 1: "g"});

- C,S는 요금 폭이 넓다
- 요금이 높은 사람들은 거의 죽지 않았다.

# Feature engineering

###  데이터로드
train과 test를 병합해서 사용

In [ ]:
    # train,test 데이터 로드
    train = pd.read_csv('./data/train.csv')    
    test = pd.read_csv('./data/test.csv')

    # 생존여부를 별도로 분리
    # inplace 속성으로 drop결과를 바로 적용
    targets = train.Survived
    train.drop(['Survived'], axis = 1, inplace=True)
    
    # 두 개의 데이터 병합
    # ignore_index 속성으로 train,test의 인덱스를 순차적으로 만듬
    combined = train.append(test,ignore_index=True)
    
    # PassengerId는 삭제
    combined.drop(['PassengerId'], 1, inplace=True)
    combined

In [ ]:
combined.shape

In [ ]:
combined.head()

### 탑승객 호칭 처리

- Braund, <b> Mr.</b> Owen Harris	
- Heikkinen, <b>Miss.</b> Laina
- Oliva y Ocana, <b>Dona.</b> Fermina (귀부인)
- Peter, <b>Master.</b> Michael J (도련님)

In [ ]:
def split_title(x):
    return x.split(',')[1].split('.')[0].strip()

In [ ]:
titles = data["Name"].apply(split_title).unique()
titles

몇 개의 타이틀로 정리

- Officer (장교)
- Royalty (귀족)
- Mr
- Mrs
- Miss
- Master

In [ ]:
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [ ]:
combined['Title'] = combined["Name"].apply(split_title)
#map함수를 통해 딕셔너리의 키 값과 시리즈의 인덱스 값이 같은 데이터를 찾아 변경
combined['Title'] = combined.Title.map(Title_Dictionary)

In [ ]:
combined.head()

### 나이 처리

- 시각화를 위해 중간 값으로 단순히 처리했지만, 좀 더 세분화 해서 나이를 채워보자
- 성별,선실등급,호칭으로 묶어서 평균나이를 구해보자

In [ ]:
# as_index 속성으로 그룹을 묶는 컬럼을 인덱스에서 제외하자
grouped_train = combined.iloc[:891].groupby(['Sex','Pclass','Title'],as_index = False)
grouped_median_train = grouped_train.median()
grouped_median_train = grouped_median_train[['Sex', 'Pclass', 'Title', 'Age']]
grouped_median_train

성별과 선실등급 그리고 호칭에 따라 평균 나이가 조금씩 다르다.

In [ ]:
def fill_age(row):
    condition = (
            (grouped_median_train['Sex'] == row['Sex']) & 
            (grouped_median_train['Title'] == row['Title']) & 
            (grouped_median_train['Pclass'] == row['Pclass'])
        ) 
    if np.isnan(row['Age']): 
        return grouped_median_train[condition]['Age'].values[0]
    else :
        return row['Age']

In [ ]:
combined['Age'] = combined.apply(fill_age, axis=1)

### 이름 처리
- 이름 특성 삭제
- 카테고리화 되어있는 호칭을 Model이 계산 할 수 있도록 one-hot-encoding

In [ ]:
#이름 특성 삭제
combined.drop('Name', axis=1, inplace=True)

In [ ]:
titles_dummies = pd.get_dummies(combined['Title'], prefix='Title')
titles_dummies.head()

In [ ]:
combined = pd.concat([combined, titles_dummies], axis=1)
combined.drop('Title', axis=1, inplace=True)

In [ ]:
combined.head()

### 요금 처리

In [ ]:
combined.Fare.fillna(combined.Fare.mean(), inplace=True)

### 승선항 처리

- 결측치는 많은 사람들이 탑승한 S로 채운다
- encoding

In [ ]:
combined.iloc[:891].Embarked.value_counts()

In [ ]:
# 결측치 처리
combined.Embarked.fillna('S', inplace=True)
    
# one-hot-encoding
embarked_dummies = pd.get_dummies(combined['Embarked'], prefix='Embarked')
combined = pd.concat([combined, embarked_dummies], axis=1)

# 승선항 특성 삭제
combined.drop('Embarked', axis=1, inplace=True)

In [ ]:
combined.head()

### 객실번호 
- 결측치는 U(Uknown)로 대체
- 숫자를 제거한 맨 앞 글자로 변경
- encoding

In [ ]:
# 결측치는 U로 대체
combined['Cabin'].fillna('U',inplace=True)

In [ ]:
# Cabin의 첫 글자로 변경
combined['Cabin'] = combined['Cabin'].str[0]

In [ ]:
# one-hot-encoding
cabin_dummies = pd.get_dummies(combined['Cabin'], prefix='Cabin')    
combined = pd.concat([combined, cabin_dummies], axis=1)

# Cabin 특성 삭제
combined.drop('Cabin', axis=1, inplace=True)

In [ ]:
combined.head()

### 성별 처리

In [ ]:
combined['Sex'] = combined['Sex'].map({'male':1, 'female':0})

남성은 1 여성은 0으로 변경

In [ ]:
combined.head()

### 객실등급 처리

In [ ]:
# one-hot-encoding
pclass_dummies = pd.get_dummies(combined['Pclass'], prefix="Pclass")
combined = pd.concat([combined, pclass_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('Pclass',axis=1,inplace=True)

In [ ]:
combined.head()

### 티켓 처리

In [ ]:
def cleanTicket(ticket):
    # .과/를 없애준다.
    ticket = ticket.replace('.', '')
    ticket = ticket.replace('/', '')
    # 공백 기준으로 자른다.
    ticket = ticket.split()
    print(ticket)
    # 자른 리스트의 각 항목의 양쪽 공백을 없애준다.
    ticket = map(lambda t : t.strip(), ticket)
    # 숫자가 아닌 것만 필터링해서 리스트로 만듬
    ticket = list(filter(lambda t : not t.isdigit(), ticket))
    if len(ticket) > 0:
        return ticket[0]
    else: 
        return 'XXX' #티켓 글자가 없으면 XXX로 표시

In [ ]:
combined['Ticket'] = combined['Ticket'].map(cleanTicket)
tickets_dummies = pd.get_dummies(combined['Ticket'], prefix='Ticket')
combined = pd.concat([combined, tickets_dummies], axis=1)
combined.drop('Ticket', inplace=True, axis=1)

In [ ]:
combined.head()

### 가족관련 특성 처리
- 부모,자녀,배우자,형제 모두 합친 특성을 새롭게 만듬
- 가족 숫자에 따라 1인, 소규모 가족, 대규모 가족으로 구분

In [ ]:
# 본인을 포함하여 모든 가족수 특성 생성
combined['FamilySize'] = combined['Parch'] + combined['SibSp'] + 1

# map함수는 apply처럼 함수를 넣어서 사용가능
# lambda는 간단한 함수를 줄여서 쓰는 방식
combined['Singleton'] = combined['FamilySize'].map(lambda s: 1 if s == 1 else 0)
combined['SmallFamily'] = combined['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
combined['LargeFamily'] = combined['FamilySize'].map(lambda s: 1 if 5 <= s else 0)

In [ ]:
combined.head()

#### 앙상블
- 단일모델 예측결과를 넘어보기 위해서 고안된 방법(단일모델: 제일 똑똑한 모델)
- 여러 모델의 결과를 종합해서 예측결과를 개선시키는 방법

#### 앙상블 종류
- Voting: 다양한 알고리즘의 모델 의견을 투표/평균내는 것(tree, knn, linear 등의 모델 종합)
    - sklearn 구현 클래스 : VotingClassifier, VotingRegressor
- Bagging: 한 개의 알고리즘 모델을 사용
    - RandomForest : tree기반  bagging 모델
    - 똑같은 알고리즘에 똑같은 데이터를 쓰기 때문에 의견이 같아질 우려가 있다
    - 그래서 다양한 의견을 내기 위한 방법을 적용
        1. 데이터 샘플의 다양화(행: row)
            - 부트스트래핑_랜덤으로 추출, 중복을 허용
        2. 데이터 특성의 다양화(열: column)
- Boosting
    - 동일한 알고리즘 모델을 사용
    - 이전 모델이 잘 못 판단한 데이터를 기반으로 다음 모델을 개선하는 방법
    - Xgboost, Catboost, LightGBM

#### Baggint 사용하기

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(random_state=77,
                                 n_estimators=3,  # tree의 개수: 나무를 몇 개 만들 것인지
                                 max_depth=10,
                                 min_samples_split=7, # 질문으로 쪼갤 수 있는 최소 데이터 갯수(7개 이상인것만 쪼개기 가능)
                                 max_features= 0.5 # 랜덤하게 선택될 feature의 비율(50%씩 컬럼을 골라서 tree 만들기)
                                 ) 

##### GridSearch
- 하이퍼파라미터의 모든 조합을 테스트->탐색 

In [ ]:
from sklearn.model_selection import GridSearchCV # cv:cross validation 교차검증

In [ ]:
m = RandomForestClassifier(random_state=77)
grid = GridSearchCV(m, # 튜닝할 모델
                    grid_param, # 튜닝할 파라미터 조합
                    cv = 5, #교차검증횟수
                    n_jobs=-1
                   )

In [ ]:
# 데이터 분리
X_train = combined.iloc[:891, :]
X_test = combined.iloc[891: , :]
y_train = targets

In [ ]:
grid.fit(X_train, y_train)

##### 제일 괜찮은 조합 살펴보기


In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_ # 제일 좋은 조합으로 학습완료된 모델

In [ ]:
pre = best_model.predict(X_test)

In [ ]:
submission = pd.read_csv('./data/gender_submission.csv')
submission['Survived'] = pre
submission.to_csv('rf_pre01.csv', index = False)

#### 교차검증 결과확인

In [ ]:
grid.cv_results_['params']

In [ ]:
grid.cv_results_['rank_test_score']  # 몇 번째 랭킹인지 확인

In [ ]:
grid.cv_results_['mean_test_score']

In [ ]:
cv_result_df = pd.DataFrame(grid.cv_results_['params'])
cv_result_df

# pd.set_option('display.max_rows',None): 데이터프레임 최대개수 해제하는거 위에서 입력

In [ ]:
cv_result_df['rank'] = grid.cv_results_['rank_test_score']
cv_result_df['score'] = grid.cv_results_['mean_test_score']    

In [ ]:
cv_result_df.sort_values(by='rank')

##### tree계열 모델의 특성 중요도 확인
- 지니불순도 알고리즘을 통해서 특성의 중요도를 측정 .feature_importances_

In [ ]:
best_model.feature_importances_

In [ ]:
X_train.columns   # 숫자값, 컬럼과 매치됨(best_model.feature_importances_/ X_train.columns )

In [ ]:
fi_df = pd.DataFrame(best_model.feature_importances_,
                    index=X_train.columns,
                    columns=['feature_importances'])
fi_df

In [ ]:
fi_df.sort_values(by='feature_importances', ascending =False).head(20)

# 예측을 미치는데 가장 중요하게 미친 컬럼들 나열(부동소수점표기방식 Title_Mr	:0.2247637)

In [ ]:
fn = fi_df.sort_values(by='feature_importances', ascending =False).head(20).index

In [ ]:
X_train[fn]   #  중요도 중심으로 추려서 성능을 다시 개선

In [ ]:
X_train_reduced = X_train[fn]
X_test_reduced = X_test[fn]

##### 주요한 특성만 추려서 GridSearch 실습

In [ ]:
grid.fit(X_train_reduced.y_train)